This is a tutorial for the DFODE-kit package.

In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

from dfode_kit.df_interface import (
    OneDFreelyPropagatingFlameConfig,
    setup_one_d_flame_case,
    df_to_h5,
)
from dfode_kit.data_operations import touch_h5, get_TPY_from_h5, random_perturb

DFODE_ROOT = os.environ['DFODE_ROOT']

### A brief introduction to the DFODE method

#### Low-dimensional manifold sampling

A key challenge in preparing training data is achieving sufficient coverage of the relevant thermochemical composition space, which is often prohibitively high-dimensional when detailed chemistry involves tens to hundreds of species. 

To address this, DFODE-kit adopts a low-dimensional
manifold sampling strategy, where thermochemical states are extracted from canonical flame configurations that retain the essential topology of high-dimensional turbulent flames. This approach ensures both computational efficiency and physical representativeness of the training datasets.

In this tutorial, we will demonstrate how to use DFODE-kit to sample a low-dimensional manifold of thermochemical states from a one-dimensional laminar freely propagating flame simulated with DeepFlame. The following code block could also be found in `case_init.ipynb` files within the case templates provides in the `cases` directory. It is used to initialize the simulation and update the dictionary files for the simulation.

In [2]:
# Operating condition settings
config_dict = {
    "mechanism": f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    "T0": 300,
    "p0": 101325,
    "fuel": "H2:1",
    "oxidizer": "O2:0.21,N2:0.79",
    "eq_ratio": 1.0,
}
config = OneDFreelyPropagatingFlameConfig(**config_dict)

# Simulation settings
settings = {
    "sim_time_step": 1e-6,
    "sim_write_interval": 1e-5,
    "num_output_steps": 100,
}
config.update_config(settings)

# Setup the case and update dictionary files
setup_one_d_flame_case(config, '.')

Solving premixed flame...
Laminar Flame Speed      :   2.3489328863 m/s
Laminar Flame Thickness  :   0.0003694362 m
One-dimensional flame case setup completed at: /data1/kexiao/projects/dfode_project/DFODE-kit/tutorials/oneD_freely_propagating_flame


Note that at the point, the simulation is not yet started. The user would need to ensure a working version of DeepFlame is available and run the `Allrun` script from command line to start the simulation.

```bash
./Allrun
```

After the simulation is completed, we proceed to use DFODE-kit to gather and manage the thermochemical data.

In [3]:
df_to_h5(
    root_dir=f"{DFODE_ROOT}/tutorials/oneD_freely_propagating_flame",
    mechanism=f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    hdf5_file_path=f"{DFODE_ROOT}/tutorials/oneD_freely_propagating_flame/tutorial_data.h5",
    include_mesh=True,
)

# The above is equivalent to the following cli command:
# dfode-kit sample --mech Burke2012_s9r23.yaml \
#     --case tutorials/oneD_freely_propagating_flame \
#     --save tutorials/oneD_freely_propagating_flame/tutorial_data.h5 \
#     --include-mesh

Species names: ['T', 'p', 'H', 'H2', 'O', 'OH', 'H2O', 'O2', 'HO2', 'H2O2', 'N2']
Saved concatenated arrays to /data1/kexiao/projects/dfode_project/DFODE-kit/tutorials/oneD_freely_propagating_flame/tutorial_data.h5


Checking the contents of the h5 file

In [4]:
touch_h5("tutorial_data.h5")

Inspecting HDF5 file: tutorial_data.h5

Metadata in the HDF5 file:
mechanism: /data1/kexiao/projects/dfode_project/DFODE-kit/mechanisms/Burke2012_s9r23.yaml
root_directory: /data1/kexiao/projects/dfode_project/DFODE-kit/tutorials/oneD_freely_propagating_flame
species_names: ['T' 'p' 'H' 'H2' 'O' 'OH' 'H2O' 'O2' 'HO2' 'H2O2' 'N2']

Groups and datasets in the HDF5 file:
Group: mesh
  Dataset: Cx, Shape: (500, 1)
  Dataset: Cy, Shape: (500, 1)
  Dataset: Cz, Shape: (500, 1)
  Dataset: V, Shape: (500, 1)
Group: scalar_fields
  Dataset: 0.0001, Shape: (500, 11)
  Dataset: 0.00011, Shape: (500, 11)
  Dataset: 0.00012, Shape: (500, 11)
  Dataset: 0.00013, Shape: (500, 11)
  Dataset: 0.00014, Shape: (500, 11)
  Dataset: 0.00015, Shape: (500, 11)
  Dataset: 0.00016, Shape: (500, 11)
  Dataset: 0.00017, Shape: (500, 11)
  Dataset: 0.00018, Shape: (500, 11)
  Dataset: 0.00019, Shape: (500, 11)
  Dataset: 0.0002, Shape: (500, 11)
  Dataset: 0.00021, Shape: (500, 11)
  Dataset: 0.00022, Shape: (500

#### Data augmentation and labeling

While laminar canonical flames provide fundamental thermochemical states,their trajectory-aligned sampling in composition space poses significant limitations for a posteriori modeling applications. First, these sampled states are confined to predefined flamelet manifolds, making the trained model highly sensitive to perturbations and leading to an over-constrained representation. Second, the sampled states span a lower-dimensional subspace, which fails to encompass the full range of thermochemical variations encountered in turbulent combustion. As a result, the model becomes vulnerable to off-manifold perturbations—deviations from the training manifold that frequently arise in turbulent reacting flows.

To tackle this challenge, a data augmentation strategy is employed, where collected states are perturbed to simulate the effects of multi-dimensional transport and turbulence disturbances.

In [5]:
thermochemical_data = get_TPY_from_h5("tutorial_data.h5")
print(thermochemical_data[0])

aug_thermochemical_data = random_perturb(thermochemical_data)
print(aug_thermochemical_data[0])

np.save("tutorial_data_aug.npy", aug_thermochemical_data)

# The above is equivalent to the following cli command:
# dfode-kit augment --h5_file tutorial_data.h5 \
#     --output_file tutorial_data_aug.npy

Number of datasets in scalar_fields group: 101
[3.00023e+02 1.02883e+05 1.50501e-41 2.85116e-02 1.63881e-47 9.68647e-48
 8.28493e-48 2.26269e-01 5.70836e-38 2.00401e-52 7.45219e-01]
[2.86602088e+02 1.02979382e+05 2.20858138e-42 2.06611700e-02
 4.98106786e-49 4.41905979e-50 6.74487245e-52 2.34119830e-01
 1.36522580e-41 2.53278147e-54 7.45219000e-01]


The CVODE integrator from Cantera is used for time integration and to provide supervised learning labels.

In [6]:
thermochemical_data = get_TPY_from_h5("tutorial_data.h5")
print(thermochemical_data[0])

aug_thermochemical_data = random_perturb(thermochemical_data)
print(aug_thermochemical_data[0])

np.save("tutorial_data_aug.npy", aug_thermochemical_data)

# The above is equivalent to the following cli command:
# dfode-kit label --mech MECH \
#     --time TIME
#     --source SOURCE 
#     --save SAVE

Number of datasets in scalar_fields group: 101
[3.00023e+02 1.02883e+05 1.50501e-41 2.85116e-02 1.63881e-47 9.68647e-48
 8.28493e-48 2.26269e-01 5.70836e-38 2.00401e-52 7.45219e-01]
[3.58226294e+02 1.02713988e+05 1.83370682e-44 3.78931118e-02
 2.36019454e-43 2.76274685e-50 7.70768939e-51 2.16887888e-01
 8.11918735e-35 8.00910288e-50 7.45219000e-01]


#### Model training

#### Model testing

Model testing is closely associated with a specific testing dataset, enabling the evaluation of the trained models’ performance. DFODE-kit can directly operate on HDF5 files, adhering to a predefined format that facilitates seamless data integration.